# Keras Tutorial
This Jupyter notebook uses documentation from Keras to provide a quick, hands-on introduction to this high-level machine learning API.  For reference, we used code from the following resource for the examples below: https://keras.io/.

# Installation and Imports

In [ ]:
!pip install keras
import keras

# OR
from tensorflow import keras

from keras.models import Sequential

# Import different layers
from keras.layers import Dense

## Core Data Structure in Keras: The Model
The core data structure in Keras is the model class, which has methods for compiling with an optimizer and loss function, fitting (training), evaluation (testing), and prediction.

# Feedforward, Stacked Models from Sequential() Class

In [ ]:
model = Sequential()  # Enables for stacking of layers
model.add(Dense(units=64, activation='relu', input_dim=100))
model.add(Dense(units=10, activation='softmax'))

## Subclassing Model() Class for Fully-Customizable Models

In [ ]:
import keras

class SimpleMLP(keras.Model):

    def __init__(self, use_bn=False, use_dp=False, num_classes=10):
        super(SimpleMLP, self).__init__(name='mlp')
        self.use_bn = use_bn
        self.use_dp = use_dp
        self.num_classes = num_classes

        self.dense1 = keras.layers.Dense(32, activation='relu')
        self.dense2 = keras.layers.Dense(num_classes, activation='softmax')
        if self.use_dp:
            self.dp = keras.layers.Dropout(0.5)
        if self.use_bn:
            self.bn = keras.layers.BatchNormalization(axis=-1)

    def call(self, inputs):
        x = self.dense1(inputs)
        if self.use_dp:
            x = self.dp(x)
        if self.use_bn:
            x = self.bn(x)
        return self.dense2(x)

model = SimpleMLP()
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])
# x_train and y_train are numpy arrays
model.fit(x_train, y_train, epochs=5, batch_size=32)

## Training and Evaluation

In [ ]:
# Define new model and add layers
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=100))
model.add(Dense(units=10, activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

# Fit model to data (x_train and y_train in this case) - data inputs need only be numpy arrays
model.fit(x_train, y_train, epochs=5, batch_size=32)

# Evaluate
loss_and_metrics = model.evaluate(x_test, y_test, batch_size=128)

# Predictions
classes = model.predict(x_test, batch_size=128)


## Saving and Loading Models

In [ ]:
# Instantiate model
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=100))
model.add(Dense(units=10, activation='softmax'))

# Save model to HDF5 file
model.save_weights(filepath)
model.load_weights(filepath, by_name=False)  # Can load for specific layers (e.g. transfer learning??) 
                                             #  by setting by_name arg to True

# Save as JSON representation
json_string = model.to_json()
model_reloaded = model_from_json(json_string)

# Save as yaml representation
yaml_string = model.to_yaml()
model_reloaded = model_from_yaml(yaml_string)

## Get Model Specs

In [ ]:
def get_model_specs(model):
    print(model.layers)  # Flattened list of tensors comprising model
    print(model.inputs)  # List of input tensors to model
    print(model.outputs)  # List of output tensors of model 
    print(model.summary())  # Brief summary of your Keras model
    print(model.get_config())  # Dict containing configuration of model

## Get and Set Weights

In [ ]:
model.get_weights()  # Returns weights of model
model.set_weights(weights)  # Sets weights of model to be weights arg

## Example of Loading Keras Datasets

In [ ]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

## Parameters for "Core" Keras Layers

1. units: Positive integer, dimensionality of the output space.
2. activation: Activation function to use (see activations). If you don't specify anything, no activation is applied (ie. "linear" activation: a(x) = x).
3. use_bias: Boolean, whether the layer uses a bias vector.
4. kernel_initializer: Initializer for the kernel weights matrix (see initializers).
5. bias_initializer: Initializer for the bias vector (see initializers).
6. kernel_regularizer: Regularizer function applied to the kernel weights matrix (see regularizer).
7. bias_regularizer: Regularizer function applied to the bias vector (see regularizer).
8. activity_regularizer: Regularizer function applied to the output of the layer (its "activation"). (see regularizer).
9. kernel_constraint: Constraint function applied to the kernel weights matrix (see constraints).
10. bias_constraint: Constraint function applied to the bias vector (see constraints).

## Important Keras Layers

1. **Dense**: Fully connected layer
2. **Activation**: Layer for applying an activation function to an output.  
3. **Dropout**: Applies a dropout layer.  Probability of dropout is an argument in layer init.
4. **Flatten**: Layer typically used for turning 2D/3D into 1D vector (e.g. Conv layers to Dense).
5. **Conv1D**: 1D convolutional layer
6. **Conv2D**: 2D convolutional layer
7. **MaxPooling1D**: Layer for max pooling in 1D
8. **MaxPooling2D**: Layer for max pooling in 2D
9. **RNN & GRU**: Base classes for recurrent layers
10. **BatchNormalization**: Batch normalization layer 

## Full Example - 1D CNN for Text Classification

In [ ]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb

# set parameters:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))